## neo4j-contrib:neo4j-spark-connector versions
https://mvnrepository.com/artifact/neo4j-contrib/neo4j-spark-connector

In [1]:
%%init_spark
# Configure Spark to use a local master
launcher.master = "local[*]"

# needs java 1.8
# launcher.packages = "neo4j-contrib:neo4j-spark-connector:2.4.0-M6"

# needs java 1.8
# launcher.packages = "neo4j-contrib:neo4j-spark-connector:2.2.1-M5"

# saveGraph mixes up id and number
launcher.packages = "neo4j-contrib:neo4j-spark-connector:2.1.0-M4"

# saveGraph mixes up id and number
# launcher.packages = "neo4j-contrib:neo4j-spark-connector:2.1.0-M3"

# doesnt have neo.saveGraph()
# launcher.packages = "neo4j-contrib:neo4j-spark-connector:2.0.0-M2" 

# doesnt do neo = Neo4j(sc)
# launcher.packages = "neo4j-contrib:neo4j-spark-connector:2.0.0-M1"  

In [2]:
println(sc.master)
// val rdd = sc.parallelize(0 to 999)
// rdd.takeSample(false, 5)

Intitializing Scala interpreter ...

Spark Web UI available at http://172.18.0.7:4040
SparkContext available as 'sc' (version = 2.0.0, master = local[*], app id = local-1551948518108)
SparkSession available as 'spark'


local[*]


In [3]:
import org.neo4j.spark._

val neo = Neo4j(sc)

import org.apache.spark.graphx._
import org.apache.spark.graphx.lib._

import org.neo4j.spark._
neo: org.neo4j.spark.Neo4j = org.neo4j.spark.Neo4j@5d27d30b
import org.apache.spark.graphx._
import org.apache.spark.graphx.lib._


In [4]:
// load graph via Cypher query
// val graphQuery = "MATCH (n:Person)-[r:KNOWS]->(m:Person) RETURN id(n) as source, id(m) as target, type(r) as value SKIP {_skip} LIMIT {_limit}"
// val graph: Graph[Long, String] = neo.rels(graphQuery).partitions(7).batch(200).loadGraph
val graphQuery = "MATCH (n:Person)-[r:KNOWS]->(m:Person) RETURN n.number as source, m.number as target, type(r) as value"
val graph: Graph[Long, String] = neo.rels(graphQuery).partitions(2).batch(15).loadGraph
println(graph.vertices.count)
println(graph.edges.count)

10
40


graphQuery: String = MATCH (n:Person)-[r:KNOWS]->(m:Person) RETURN n.number as source, m.number as target, type(r) as value
graph: org.apache.spark.graphx.Graph[Long,String] = org.apache.spark.graphx.impl.GraphImpl@5e29a238


In [5]:
// val graph2_pagerank = PageRank.run(graph2,500)
val graph_pagerank = PageRank.run(graph,10)

graph_pagerank: org.apache.spark.graphx.Graph[Double,Double] = org.apache.spark.graphx.impl.GraphImpl@fa579ba


In [6]:
graph.edges.collect()

res2: Array[org.apache.spark.graphx.Edge[String]] = Array(Edge(1,3,KNOWS), Edge(1,4,KNOWS), Edge(2,4,KNOWS), Edge(2,5,KNOWS), Edge(3,5,KNOWS), Edge(3,6,KNOWS), Edge(4,6,KNOWS), Edge(4,7,KNOWS), Edge(5,7,KNOWS), Edge(5,8,KNOWS), Edge(6,8,KNOWS), Edge(6,9,KNOWS), Edge(7,9,KNOWS), Edge(7,10,KNOWS), Edge(8,1,KNOWS), Edge(8,10,KNOWS), Edge(9,1,KNOWS), Edge(9,2,KNOWS), Edge(10,2,KNOWS), Edge(10,3,KNOWS), Edge(1,3,KNOWS), Edge(1,4,KNOWS), Edge(2,4,KNOWS), Edge(2,5,KNOWS), Edge(3,5,KNOWS), Edge(3,6,KNOWS), Edge(4,6,KNOWS), Edge(4,7,KNOWS), Edge(5,7,KNOWS), Edge(5,8,KNOWS), Edge(6,8,KNOWS), Edge(6,9,KNOWS), Edge(7,9,KNOWS), Edge(7,10,KNOWS), Edge(8,1,KNOWS), Edge(8,10,KNOWS), Edge(9,1,KNOWS), Edge(9,2,KNOWS), Edge(10,2,KNOWS), Edge(10,3,KNOWS))


In [7]:
graph.vertices.collect()

res3: Array[(org.apache.spark.graphx.VertexId, Long)] = Array((4,7), (6,9), (8,1), (10,2), (2,5), (1,3), (3,6), (7,9), (9,1), (5,8))


In [8]:
graph_pagerank.edges.collect()

res4: Array[org.apache.spark.graphx.Edge[Double]] = Array(Edge(1,3,0.25), Edge(1,4,0.25), Edge(2,4,0.25), Edge(2,5,0.25), Edge(3,5,0.25), Edge(3,6,0.25), Edge(4,6,0.25), Edge(4,7,0.25), Edge(5,7,0.25), Edge(5,8,0.25), Edge(6,8,0.25), Edge(6,9,0.25), Edge(7,9,0.25), Edge(7,10,0.25), Edge(8,1,0.25), Edge(8,10,0.25), Edge(9,1,0.25), Edge(9,2,0.25), Edge(10,2,0.25), Edge(10,3,0.25), Edge(1,3,0.25), Edge(1,4,0.25), Edge(2,4,0.25), Edge(2,5,0.25), Edge(3,5,0.25), Edge(3,6,0.25), Edge(4,6,0.25), Edge(4,7,0.25), Edge(5,7,0.25), Edge(5,8,0.25), Edge(6,8,0.25), Edge(6,9,0.25), Edge(7,9,0.25), Edge(7,10,0.25), Edge(8,1,0.25), Edge(8,10,0.25), Edge(9,1,0.25), Edge(9,2,0.25), Edge(10,2,0.25), Edge(10,3,0.25))


In [9]:
graph_pagerank.vertices.collect()

res5: Array[(org.apache.spark.graphx.VertexId, Double)] = Array((4,0.8326567563103858), (6,0.8326567563103858), (8,0.8326567563103858), (10,0.8326567563103858), (2,0.8326567563103858), (1,0.8326567563103858), (3,0.8326567563103858), (7,0.8326567563103858), (9,0.8326567563103858), (5,0.8326567563103858))


In [ ]:
--

In [ ]:
neo.saveGraph?

In [10]:
val mypattern_cmon = neo.pattern(("Person","number"),("KNOWS",null),("Person","number"))

mypattern_cmon: org.neo4j.spark.Neo4j = org.neo4j.spark.Neo4j@5d27d30b


In [11]:
// uses pattern from above to save the data, merge parameter is false by default, only update existing nodes
neo.saveGraph(graph_pagerank, "rankcmon", mypattern_cmon.pattern, merge = true)

res6: Long = 120


In [12]:
// uses pattern from above to save the data, merge parameter is false by default, only update existing nodes
neo.saveGraph(graph_pagerank, "rankcmon")

res7: Long = 120


In [ ]:
// uses pattern from parameter to save the data, merge = true also create new nodes and relationships
neo.saveGraph(graph, "rank",neo.pattern(("Person","id"),("FRIEND","years"),("Person","id")), merge = true)